# A quick intro on RNASeq analysis

(work in progress...)

github: rodriguezmDNA

### 1. Quality Control on Raw Data.

* This snippet gets library sizes for each file (number of reads in each library).

> ``for n in RawData/*.fastq.gz; do touch rawLibSize.txt; tmp=`basename $n`; tmp=${tmp%.fastq.gz}; lNum=`gunzip -dc $n| grep "@" | wc -l`; echo $tmp $lNum >> rawLibSize.txt;done``


Run fastqc on the data.

> ``fastqc RawData/* --noextract --outdir 00a_qcRaw/``


### 2. Data preprocessing.

During RNASeq library preparation artificial DNA sequences are introduced such as adapters (for the sequencing process), barcodes (for multiplexing). Other sources of contamination can be sequencing of primer dimers.   

**1 Barcode removal.** For this case barcodes have to be manually removed. <br>
`fastx_trimmer` will be to remove the first 8bp. These appear due to the need for multiplexing. <br>

Parameters used: 

> ``-v -f 9 -Q33``

* -v: Verbose
* -f 9: First base to keep. This means first 8 bases are discarded.
* -Q 33: Only keep sequences with a min. quality (phred) score (33) 

<hr>

**On adapters.**

3' adapters must be trimmed to avoid introducing errors in subsequent RNAseq analysis (namely, during read alignment to the reference genome/transcriptome). If reads are long (>100bp) this step can be skipped.

However for short reads (ie, 50bp) it is recommended to remove these adapter sequences.

A useful tool to infer/test for adapter sequence in the data is `minion` from the Kraken suite. In short, it takes a FASTQ file and reads the first 2 million reads. It will try to infer the presence of 3' adapters or compare to a known (given) sequence to test for its presence.


Test for the presence of an adapter. **

`> minion search-adapter -i RawData/Col-1-1_S109_L006_R1_001.fastq.gz -adapter ATCTCGTATGCCGTCTTCTGCTTG`

We can also visually se the adapter's location in the reads using `grep`: <br>

`> gunzip -dc RawData/Col-1-1_S109_L006_R1_001.fastq.gz | grep "ATCTCGTATGCCGTCTTCTGCTTG" | wc -l` 

Finally, if we don't know (or are unsure about) the sequence of the adapter, minion can try to infer it for us:

`> minion search-adapter -i RawData/Col-1-1_S109_L006_R1_001.fastq.gz`



** List of probable adapters **

* ATCTCGTATGCCGTCTTCTGCTTG (3' Small RNA)
* CACACGTCTGAACTCCAG
* GTGACTGGAGTTCAGACGTGTGCTCTTCCGATC (From the BRAD-Seq paper, matches TruSeq Ribo Profile)
* AGATCGGAAGAGCACACGTCT  (TruSeq Ribo Profile, 3' Adapter)
* AATGATACGGCGACCACCGAGATCTACACTCTTTCCCTACACGACGCTCTTCCGATCT Universal Adapter


\** The adapter was taken from a list of [Illumina adapter sequences](https://support.illumina.com/content/dam/illumina-support/documents/documentation/chemistry_documentation/experiment-design/illumina-adapter-sequences_1000000002694-01.pdf) and corresponds to the _Small RNA 3' Adapter_ sequence.

<hr>



**2 Trim reads.**
There are many approaches to trim reads (trimmomatic, fastx_clipper). The one I find more comprehensive is `Reaper`, also from the Kraken suite.

Logic: Decompress file to stdout, remove first 8bp (barcode) with fastx_trimmer, use reaper to remove the adapter sequence. 

Test differnt adapter sequences, ie:
> ``seqAdapt="AATGATACGGCGACCACCGAGATCTACACTCTTTCCCTACACGACGCTCTTCCGATCT"`` <br>

<hr>

I used this one:
> ``seqAdapt="AGATCGGAAGAGCACACGTCT"`` <br> 

Try this one: <br>  
tabu="GATCGGAAGAGCACACGTCTGAACTCCAGTCAC" #5'Adapter side from TruSeq Illumina
seqAdapt="AGATCGGAAGAGCACACGTCT" #3' Adapter from Illumina
-- Results: Looks good, kmer content seems better distributed, not at the start of the read like previous.

Or this one
tabu="GATCGGAAGAGCACACGTCTGAACTCCAGTCAC"
seqAdapt="ATCTCGTATGCCGTCTTCTGCTTG"        
-- Results: Not as good as the previous one. More Kmers at the center.


Or this one:
> ``tabu="ATCTCGTATGCCGTCTTCTGCTTG"
> seqAdapt="AGATCGGAAGAGCACACGTCT" #3' Adapter from Illumina``

<br>

> ``tabu="CAAGCAGAAGACGGCATACGAGAT"
> seqAdapt="GTGACTGGAGTTCAGACGTGTGCTCTTCCGATC" #From BRADseq paper``


Use this code for trimming barcode and adapters. 


> ``for n in RawData/Col-1-1*.fastq.gz; do tmp=`basename $n`; tmp=${tmp%.fastq.gz}; echo Doing $tmp; gunzip -dc $n | fastx_trimmer -v -f 9 -Q33 - | reaper -geom no-bc -tabu $tabu -3pa $seqAdapt -dust-suffix 6/ACTG -dust-suffix-late 6/ACTG -nnn-check 1/1 -qqq-check 35/10 -clean-length 30 --bcq-late -tri 20 -polya 5 --noqc -basename 01_trimmed/$tmp ;done``


### 3. Quality Control after trimming.

* This snippet gets library sizes for each file (number of reads in each library).

> ``for n in 01_trimmed/*.lane.clean.gz; do touch cleanLibSize.txt; tmp=`basename $n`; tmp=${tmp%.fastq.gz}; lNum=`gunzip -dc $n| grep "@" | wc -l`; echo $tmp $lNum >> cleanLibSize.txt;done``


Run fastqc on the data.

> ``fastqc 01_trimmed/*.lane.clean.gz --noextract --outdir 00b_qcClean/``



### Mapping

Now it's time to align the reads to a reference. Whenever possible, use a genome reference to align. Alignment algorithms are optimized to perform better when aligning to the whole genome rather than the transcriptome.


##### Bowtie testing

Bowtie is one of the most widespread used aligners. It's fast and sensitive and can perform well on a desktop computer (unlike some mappers like STAR which requires ~30GB in RAM).

* First we build the index:

> ``bowtie-build meta/genome_ath_TAIRv10.fa athGenome/athbw1``


Try different parameters:

### Mapping

Parameters used by Brad:
> ``bowtie -a --best --strata -n 1 -m 1 -p 4 --sam --norc --tryhard $db $ARGV[0]  BOWTIEoutput_NewComp7M_SS_m1_ITAG500+500/$basename.sam ``

* Normal bowtie mapping
> `` for each in test/*.fq; do tmp=`basename $each`; tmp=${tmp%.fq}; echo $tmp; bowtie -a --best --strata -n 1 -m 1 -p 4 --sam --norc --tryhard genomeIndex/ath $each $tmp.sam 2>> log/$tmp.log; done``

* Pass to samtools, filter by quality of mapping. Save as bam.

> ``bowtie -a --best --strata -n 1 -m 1 -p 4 --sam --norc --tryhard genomeIndex/ath test/test.fq - 2> test.log | samtools view -h -q30 -b -S -o output.bam ``

<hr>

__Use this one:__

> ``for each in 01_preProcessed/*.fq; do tmp=`basename $each`; tmp=${tmp%.fq}; echo $tmp; bowtie -a --best --strata -n 1 -m 1 -p 4 --sam --norc --tryhard genomeIndex/ath $each 2> logs/bowtielog/$tmp.log | samtools view -q30 -bS - -o 02_aligned/$tmp.bam; done``

##-q30 ensures only the best alignment will be reported

* Pass bowtie output in SAM format to samtools, sort and store as bam:

> ``for each in 02_aligned/*.bam; do tmp=`basename $each`; tmp=${tmp%.bam}; echo $tmp; samtools sort $each 02_aligned/sorted/$tmp.sorted; done``

<hr>
**Bowtie 2 ** 

Generally recommended for longer reads ( >50bp)




* Some guy tested different parameters (https://samnicholls.net/2016/12/24/bowtie2-metagenomes/) <br> 
> ``--local -D 20 -R 3 -L 3 -N 1 -p 8 --gbar 1 --mp 3`` <br>
** Note, this is too slow. **


    * From the bowtie2 manual:
    * -R and -D are effort options:
        * -D is the number of times that seed extensions (alignment attempts) can fail before giving up. **_'Increasing -D makes Bowtie 2 slower, but increases the likelihood that it will report the correct alignment for a read that aligns many places'_**
        * -R number of times bowtie2 will try to re-seed reads with repetitive seeds.
    * -L is the lenght of the seed to be aligned. For most forms of alignment in bowtie2, the default value is 20.
    * -N is the number of mismatches allowed during seed alignment.

> _Seed extension is an approach bowtie (and bowtie2) use to align a read. To narrow down the possiblities of finding a possible alignment, it takes a random substring of the read and finds a perfect match (or matches) in the index. Then extends the length of the read until it finds the region that better matches the read._


* If data is strand specific use the flag ``--norc``

* To avoid reads that didn't have a valid alignment use: ``--no-unal`` 




Try it out:
> ``bowtie2 --norc -x ../athGenome/athbw2 --local -D 20 -R 3 -L 3 -N 1 -p 8 --gbar 1 --mp 3 --no-unal SALK-012253-1-1_S110_L006_R1_001.lane.clean -S bow2.out.sam``  (**Too slow**)

The parameters are quite similar to the `--very-sensitive-local` alignment mode.
> ``bowtie2 --norc -x ../athGenome/athbw2 --local --very-sensitive-local -p 4 --no-unal SALK-012253-1-1_S110_L006_R1_001.lane.clean -S bow2.out.sam``




** About Quality of the Mapped Sequences ** 

http://davetang.org/muse/2011/09/14/mapping-qualities/

** About Adapters ** 

http://tucf-genomics.tufts.edu/documents/protocols/TUCF_Understanding_Illumina_TruSeq_Adapters.pdf

### References

* Kraken Suite. https://www.ebi.ac.uk/research/enright/software/kraken
* hannonlab.cshl.edu/fastx_toolkit/
* https://support.illumina.com/content/dam/illumina-support/documents/documentation/chemistry_documentation/experiment-design/illumina-adapter-sequences_1000000002694-01.pdf